In [1]:
#Import packages and define functions
import json
import os
from PIL import Image
from ultralytics import YOLO
import shutil
import random
import torch

if torch.cuda.is_available():
    torch.cuda.set_device(0) # Set to your desired GPU number
    print('using gpu')

home = os.path.expanduser("~")

def create_yolo_folders(path):
    """
    Creates the folder structure needed by the YOLO format.
    """

    os.mkdir(path)
    path_images = os.path.join(path, 'images')
    path_labels = os.path.join(path, 'labels')

    # Create folder structure.
    os.mkdir(path_images)
    os.mkdir(path_labels)
    os.mkdir(os.path.join(path_images, 'train'))
    os.mkdir(os.path.join(path_images, 'val'))
    os.mkdir(os.path.join(path_images, 'test'))
    os.mkdir(os.path.join(path_labels, 'train'))
    os.mkdir(os.path.join(path_labels, 'val'))
    os.mkdir(os.path.join(path_labels, 'test'))

    print(f"Created {path} directory and subdirectories.")

def convert_format_perception_to_yolo(src_folder='./solo/',
                                      dst_folder='./Converted_data/',
                                      copy_images=False,
                                      train_cutoff=70,
                                      val_cutoff=90,
                                      minVisibility=0.2):
    """
    Converts the synthetic data from the SOLO format output by the Perception package to
    the format used by YOLO. Also splits the data into a train, validation, and test set
    according to the values of dataset_size, train_cutoff, and val_cutoff.
    """

    # Create destination folders in the yolo format.
    try:
        create_yolo_folders(dst_folder)
    except OSError as error:
        print(f'Folder {dst_folder} already exists.')

    folder = 'train'
    
    # Loop over all the files.
    for (root, dirs,files) in (os.walk(src_folder)):
        dirs.sort(key=lambda directory: int(directory.split('.')[1]))

        image_nr = root.split('.')[-1]

        print(root)
        # Ignore everything except the sequence.X directories.
        if 'sequence' not in root:
            continue
        if root.endswith('sequence.0'):
            continue

        # Choose the correct folder name.
        if int(image_nr) >= train_cutoff:
            folder = 'val'
        if int(image_nr) >= val_cutoff:
            folder = 'test'

        

        # Get the images width and height. Copy the image to the YOLO images directory.
        image_path = os.path.join(root, 'step0.camera.png')
        image = Image.open(image_path, 'r')
        image_width, image_height = image.size
        image_cpy_path = os.path.join(dst_folder, 'images', folder, f'{image_nr}.png')

        if copy_images:
            image.save(image_cpy_path)
            image.close()
        else:
            image.close()
            shutil.move(image_path, image_cpy_path)

        # Read the bounding box data from the json files.
        label_path = os.path.join(root, 'step0.frame_data.json')
        with open(label_path, 'r') as f:
            example_data = json.load(f)

            

            bbox_string = ''

            try:
                values = example_data['captures'][0]['annotations'][0]['values']
                for i, bbox in enumerate(values):
                    # Calculate the center of the bounding boxes.
                    x_center = bbox['origin'][0] + 0.5 * bbox['dimension'][0]
                    y_center = bbox['origin'][1] + 0.5 * bbox['dimension'][1]

                    # Normalize the xywh as described in the YOLO Ultralytics docs:
                    # https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/
                    x = x_center / image_width
                    y = y_center / image_height
                    w = bbox['dimension'][0] / image_width
                    h = bbox['dimension'][1] / image_height
                    #remove occluded bboxes
                    if example_data['metrics'][4]['values'][i]['percentVisible'] < minVisibility:
                        continue

                    yolo_bbox = f"{bbox['labelId']} {x} {y} {w} {h}\n"
                    print(yolo_bbox)

                    bbox_string += yolo_bbox
            except KeyError:
                print('No bounding boxes')

            label_file_path = os.path.join(dst_folder, 'labels', folder, f'{image_nr}.txt')
            with open(label_file_path, 'w') as label:
                label.write(bbox_string)
    

def create_yaml(yaml_name, yolo_data,labels):
    """
    This function is used to quickly create the YOLO yaml files.
    """
    if os.path.exists(yaml_name):
        os.remove(yaml_name)

    names=''
    for l in labels:
        names+=f'    {l[0]}: {l[1]}\n'

    yaml_contents = f"""path: {yolo_data} # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/val  # val images (relative to 'path')
test: images/test # test images (optional)

names:
{names}"""

    with open(yaml_name, 'w') as yaml_file:
        yaml_file.write(yaml_contents)

    print(f'Yaml file {yaml_name} created.')


def include_bg_photos(yolo_data, 
                      background_photos=r'./Background_photos', 
                      desired_fraction = 0.05):
    """
    Includes a fraction of unlabeled background images in the dataset 
    """
    
    desired_images = (desired_fraction/(1-desired_fraction))*(len(os.listdir(f'{yolo_data}\\images\\train'))+len(os.listdir(f'{yolo_data}\\images\\val')))
    photos = os.listdir(background_photos)
    random.shuffle(photos)
    n = min(desired_images,len(photos))
    print(f'Desired amount of bg-images={desired_images}, using {n} images.')
    
    for i,(files) in enumerate(photos):
        if i < round(0.8*n):
            shutil.copy(background_photos+f'\\{files}',f'{yolo_data}\\images\\train')
        elif i < n:
            shutil.copy(background_photos+f'\\{files}',f'{yolo_data}\\images\\val')
        else:
            print(f'{i} bg photos included.')
            break

        
def get_labels(solo_folder):
    """
    Gets the labels and corresponding id from the perception data.
    """
  
    with open(solo_folder+r'\sequence.2\step0.frame_data.json', 'r') as f:
        json_data = json.load(f)
    labels = []
    
    for IDs in json_data['metrics'][3]['values']:
        id = IDs['labelId']
        label_name = IDs['labelName']
        labels.append((id,label_name))
    print(labels)
    return labels

using gpu


In [ ]:
#Data convertion

#choose dataset locations
solo_folder=r'./solo'
dataset_name='cone_1000_newRand_2'
dst=f'{home}/datasets/{dataset_name}'
background_photos=r'./Background_photos'

#prepares the dataset for yolov8 training
if not os.path.exists(home + '/datasets'):
    os.mkdir(home + '/datasets')
    
labels = get_labels(solo_folder)
with open(solo_folder+'\metadata.json', 'r') as f:
        metadata = json.load(f)
dataset_size=metadata['totalFrames']
print(f'Dataset size = {dataset_size}')

convert_format_perception_to_yolo(src_folder=solo_folder,
                                  dst_folder=dst,
                                  copy_images=False, #Copy if original solo data are to be used again
                                  train_cutoff=round(dataset_size*0.8),
                                  val_cutoff=dataset_size,
                                  minVisibility=0.2)

create_yaml(f'{dst}/{dataset_name}.yaml',
             dst,labels)

include_bg_photos(dst,
                  background_photos=background_photos, 
                  desired_fraction = 0.05)


In [25]:
#This is just to create the yaml file if the dataset is converted already

#choose dataset
dataset_name='cone_1000_newRand_2'
dst=f'{home}/datasets/{dataset_name}'

#creates yaml file with manual list of labels below
labels = [(0, 'cone')]
if not os.path.exists(home + '/datasets'):
    os.mkdir(home + '/datasets')
create_yaml(f'{dst}/{dataset_name}.yaml',
             dst,labels)

Yaml file /home/campus/datasets/cone_1000_newRand_2/cone_1000_newRand_2.yaml created.


In [6]:
# Load a model
model = YOLO('yolov8s.pt')  # load a pretrained model (recommended for training)

#choose dataset
dataset_name='cone_1000_newRand_2'
dst=f'{home}/datasets/{dataset_name}'


# Train the model
results = model.train(data=f'{dst}/{dataset_name}.yaml', 
                      epochs=35, imgsz=1280, name=dataset_name, batch=-1, cache=True)

Ultralytics YOLOv8.2.10 🚀 Python-3.12.3 torch-2.3.0+cu121 CUDA:0 (NVIDIA RTX A3000 12GB Laptop GPU, 12045MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/campus/datasets/cone_1000_newRand_2/cone_1000_newRand_2.yaml, epochs=35, time=None, patience=100, batch=-1, imgsz=1280, save=True, save_period=-1, cache=True, device=None, workers=8, project=None, name=cone_1000_newRand_23, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fa

train: Scanning /home/campus/datasets/cone_1000_newRand_2/labels/train.cache... 800 images, 62 backgrounds, 0 corrupt: 100%|██████████| 842/842 [00:00<?, ?it/s]
train: Caching images (2.2GB RAM): 100%|██████████| 842/842 [00:03<00:00, 230.16it/s]
val: Scanning /home/campus/datasets/cone_1000_newRand_2/labels/val.cache... 200 images, 14 backgrounds, 0 corrupt: 100%|██████████| 211/211 [00:00<?, ?it/s]
val: Caching images (0.6GB RAM): 100%|██████████| 211/211 [00:00<00:00, 220.58it/s]


Plotting labels to runs/detect/cone_1000_newRand_23/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0004921875), 63 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 8 dataloader workers
Logging results to runs/detect/cone_1000_newRand_23
Starting training for 35 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/35      8.18G     0.6361      1.611      1.086          7       1280: 100%|██████████| 121/121 [00:36<00:00,  3.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:02<00:00,  6.11it/s]


                   all        211        861      0.817       0.83      0.885       0.69

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      13/35      8.28G     0.4302     0.3646     0.9376         14       1280: 100%|██████████| 121/121 [00:39<00:00,  3.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:02<00:00,  6.30it/s]

                   all        211        861      0.957      0.962      0.986      0.913



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      14/35      8.28G     0.4069     0.3498     0.9268         38       1280:  93%|█████████▎| 112/121 [00:36<00:02,  3.06it/s]

In [5]:
from ultralytics import YOLO

# Load a model
model = YOLO(r's_ubuntu_640.pt')  # load a custom trained model

# Export the model
model.export(format='onnx')

Ultralytics YOLOv8.2.10 🚀 Python-3.12.3 torch-2.3.0+cu121 CPU (12th Gen Intel Core(TM) i7-12850HX)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from 's_ubuntu_640.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (21.5 MB)

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: export success ✅ 1.1s, saved as 's_ubuntu_640.onnx' (42.6 MB)

Export complete (2.8s)
Results saved to /home/campus/Documents/Pythonscript
Predict:         yolo predict task=detect model=s_ubuntu_640.onnx imgsz=640  
Validate:        yolo val task=detect model=s_ubuntu_640.onnx imgsz=640 data=/home/campus/datasets/cone_1000_newRand_2/cone_1000_newRand_2.yaml  
Visualize:       https://netron.app


's_ubuntu_640.onnx'